# Table of contents

1. [Introduction](#part1)
2. [Settings](#part2)
3. [Synthetic data](#part3)
4. [Processing data](#part4)
5. [Markov Chain Monte Carlo](#part5)
6. [Visualisation Results](#part6)   
<br>
<br>

# 1. Introduction <a class="anchor" id="part1"></a>
This notebook is an accompaniment to a research project entitled *A Markov Chain Monte Carlo approach for the estimation of photovoltaic system parameters*. In this project, we have devised a method to determine the orientation ($\phi$), tilt ($\theta$) and number of panels ($N_{p}$) or system size of photovoltaic (PV) systems using a Markov Chain Monte Carlo (MCMC) approach. We identify clear day observed profiles from the online portal PVOutput (www.pvoutput.org) and compare these to modelled profiles from the PVLib library.

# 2. Settings <a class="anchor" id="part2"></a>

It is highly recommended that you create an environment which will allow this notebook to run smoothly. This project has been tested in Linux using Python 3.7, where pyenv was used for the Python version mangement and virtualenv was used to create the environment. Please see the requirements.txt file demonstrating which packages are necessary. It may be necessary to add the Python environment kernel to the Jupyter notebook before being able to run the code in the notebook. This can be done with something like: python -m ipykernel install --name pv_mcmc. Also, don't forget to first activate the environment before running the Jupyter-notebook. This can be achieved by doing something along the following lines:
source ./.venv/bin/activate

This project has not yet been tested in any other operating system.

## Import Python libraries

In [ ]:
# Import standard Python functions
import numpy as np
import pandas as pd
import pvlib
import emcee
import os, sys
from IPython.display import Image
sys.path.append('./scripts')
# Import custom functions
import mcmc_functions_4D as mcmc
import data_processing_4D as processing
import synthetic_data_4D as synthetic
import plot_functions_4D as plot

## Set and create directories

In [ ]:
home_path = os.path.realpath('.')
os.path.isdir('new_folder')
data_path = os.path.join(home_path, 'mock_data/')
chains_path = os.path.join(home_path, 'mcmc_chains/')
plots_path = os.path.join(home_path, 'mcmc_plots/')

# Create directories if they do not already exist
paths = [data_path, chains_path, plots_path]
processing.create_directories(paths)

# 3. Synthetic or observed data <a class="anchor" id="part3"></a>
While in the original research paper, we used PVOutput data to determine PV system parameters we cannot provide these in this repository. For those interested in the original data, we refer the reader to the following Github page, which is a dedicated library for downloading PVOutput data: https://github.com/openclimatefix/pvoutput. We also provide code to read in the pvoutput data (in the format we received the data). To allow us to demonstrate our technique, we generate some synthetic data using PVLib, which we subsequently use as our observed data. The user is free to generate these data for different system specifications and/or days.

In [ ]:
# Define whether the notebook is run with synthethic or real pvoutput data. 
# This notebook will only run with the synthethic option unless you have your own copy of PVOutput data.
option = 'synthetic'


if (option == 'synthetic'):
    # Define system IDs.
    lst_ids = [1, 2, 3]

    # Define the following 5 parameters: latitude, longitude, tilt, orientation and number of panels
    sys1 = [52, 5, 20, 270, 8]
    sys2 = [54, 7, 40, 50, 24]
    sys3 = [53, 6, 60, 140, 16]
    specs = [sys1, sys2, sys3]
    meta = list(zip(lst_ids, specs))

    # Define time resolution of profiles (in minutes).
    time_resolution = '5'

    # Define the clear sky days
    lst_dates = ['20160508', '20170526', '20180505']

    # Check if files already exist. If files already exist, you will be asked whether you wish to continue: Y/N.
    #files_check = synthetic.check_files(meta, data_path)
    files_check = 'Y'
    # If yes or if the files do not exist yet: synthetic data will be created
    if files_check=='Y':
        synthetic.create_observations(meta, lst_dates, data_path, time_resolution)
        synthetic.add_noise(lst_ids, lst_dates, data_path)
else:
    # define path where the pvoutput data is stored. Modify line below to suit your needs.
    pvo_path = os.path.join(home_path, 'pvoutput_data/')
    print('skipping synthethic data creation because you have chosen to run with real data.')


# 4. Data manipulation <a class="anchor" id="part4"></a>

In [ ]:
# Retrieve meta data (lon and lat) for system
if (option == 'synthetic'):
    # Select system and date for which to perform MCMC.
    sid = 1
    date = '20160508'; year = int(date[0:4]); month = int(date[4:6]); day = int(date[6:8])

    # Define time resolution of profiles (in minutes). This need not be the same as the observed profiles.
    time_resolution = '5'
    lat, lon = processing.retrieve_meta(sid, data_path)
    
    # Obtain measurement data for system
    dict_measurement = processing.retrieve_measurements(sid, date, data_path, option)
    
elif (option == 'real'):
    # Select system and date for which to perform MCMC.
    sid = 25918
    date = '20180507'; year = int(date[0:4]); month = int(date[4:6]); day = int(date[6:8])
    
    # Define time resolution of profiles (in minutes). This need not be the same as the observed profiles.
    time_resolution = '5'
    lat, lon = processing.retrieve_meta(sid, pvo_path)
    
    # Obtain measurement data for system
    dict_measurement = processing.retrieve_measurements(sid, date, pvo_path, option)



# Define naive times
times, ymd, hm, timezone = processing.get_naive_times(date, time_resolution)

# 5. Markov Chain Monte Carlo <a class="anchor" id="part5"></a>

In [ ]:
# Determine number of walkers and iterations for MCMC
theta_var = 20; phi_var = 20; Np_var = 2; sigma_var = 1; ndim = 4; nwalkers = 40;
variation = [theta_var, phi_var, Np_var, sigma_var]

# Set guess values for theta, phi and Np, which are used for first iteration in MCMC
#theta_guess, phi_guess, Np_guess, sigma_guess = mcmc.guess_values()
theta_guess, phi_guess, Np_guess, sigma_guess = 20, 270, 8, 1
guess_values = theta_guess, phi_guess, Np_guess, sigma_guess; guess_values = np.array(guess_values)

# Determine the starting values for a Gaussian ball around the guess parameters
guess_values = mcmc.get_starting_values_ball(guess_values, ndim, nwalkers, variation)

In [ ]:
# Define the parameters for which to run the burn in.
minClusters = 1; maxClusters = 3;
# CAUTION: Define the number of processors for which to run the MCMC. Here I have chosen 8, but check how many you have available to you.
nproc = 8; 
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
iterations = 10000;

system_meta = sid, lon, lat, timezone
system_measurements = dict_measurement, times, hm
mcmc_config = nwalkers, ndim, nproc, iterations, guess_values
pvlib_databases = sandia_modules, sapm_inverters
clustering_params = minClusters, maxClusters
paths = chains_path, plots_path
sampling_state = mcmc.burnin(system_meta, system_measurements, date, mcmc_config, pvlib_databases, \
                                        clustering_params, paths)

In [ ]:
%matplotlib inline
# update the MCMC configuration for which to run the sampling, i.e. nwalkers and guess_values = sampling_state
nwalkers = 8
mcmc_config = nwalkers, ndim, nproc, iterations, sampling_state
mcmc.sampling(system_meta, system_measurements, date, mcmc_config, pvlib_databases, \
                                      clustering_params, paths)


# 6. Visualisation Results <a class="anchor" id="part5"></a>

In [ ]:
flat_samples = plot.retrieve_samples(sid, date, nwalkers, iterations, chains_path)

# Obtain the best fit values + uncertainties for the parameters
df_params, lst_params = plot.retrieve_best_params(flat_samples, ndim, 2)
display(df_params)
plot.plot_pdfs(flat_samples, sid, date, nwalkers, iterations, plots_path, lst_params)

In [ ]:
# Compare observed and modelled profiles
time_resolution = 5
print(date)
plot.observed_modelled_profiles(sid, date, time_resolution, data_path, df_params, option)

# Probability Density Functions for the three different parameters
display(Image(plots_path + 'pdfs_id%s_%s_w%s_i%s.png' %(str(sid), date, str(8), str(iterations)), width = 700, height = 700))

# Auto correlation time
display(Image(plots_path + 'autocorrelation_id%s_%s_w%s_i%s.png' %(str(sid), date, str(8), str(iterations))))

